In [1]:
import pandas as pd
import json
from pandas import json_normalize
import json

import warnings 
warnings.filterwarnings('ignore')


In [3]:
# Read Data
df21 = pd.read_parquet('data/transaction_2021.parquet')
df22 = pd.read_parquet('data/transaction_2022.parquet')
df23 = pd.read_parquet('data/transaction_2023.parquet')
df_all = pd.concat([df21, df22, df23])

In [4]:
df_all.head()

,payment_date,ent,center,product_json,total_amount,discount,paid_amount,trans_origin_type
0,2021-01-01 00:00:00.000,TCC43RQQZ3B,TKC4CJ,"[{""product"":""1ECZ"",""qty"":1},{""product"":""8CEC4Q...",1627000.0,0.0,0,online
1,2021-01-01 00:00:00.000,TCC4Q4JZ4Q0,TKCEWQ,"[{""product"":""4E4W"",""qty"":2},{""product"":""DC4CC4...",594000.0,0.0,0,online
2,2021-01-01 00:00:00.000,TCC4RJ11QE7,TKC14E,"[{""product"":""IC4CC4"",""qty"":1}]",468000.0,0.0,0,online
3,2021-01-01 00:00:00.000,TCC434QRCJY,TKC14R,"[{""product"":""5C4CC4"",""qty"":1}]",341000.0,0.0,0,online
4,2021-01-01 00:00:00.000,TCC43RRCE10,TKC11Z,"[{""product"":""5C4C4Q"",""qty"":1}]",189000.0,0.0,0,online


# Clean Data

In [5]:
# transaction paid_amount == 0
print(df_all[df_all['paid_amount'] == 0]['total_amount'].count(), "transactions")
print(df_all[df_all['paid_amount'] == 0]['total_amount'].sum() / 1000000, "million baht")

2265735 transactions
1894882.79382 million baht


In [6]:
# transaction paid_amount != 0
print(df_all[df_all['paid_amount'] != 0]['total_amount'].count(), "transactions")
print(df_all[df_all['paid_amount'] != 0]['total_amount'].sum() / 1000000, "million baht")

140581 transactions
114318.7389 million baht


extract product_json

In [7]:
# extract product_json
a = df_all["product_json"].to_list()

list = []

try:
    for i in a:
        if str(i) == "nan":
            break
        dict = json.loads(i)
        for j in dict:
            list.append(j["product"])
except:
    print(str(i))

None


In [8]:
df_product = pd.DataFrame(data=list, columns=["product"])
df_product_group = df_product.groupby(["product"]).count().reset_index()
df_product_list = df_product_group["product"].to_list()
len(df_product_list)

477

In [9]:
df_full = df_all.copy()
df_full['product'] = df_full['product_json'].apply(lambda x: [item['product'] for item in json.loads(x)] if x is not None else [])
df_full['qty'] = df_full['product_json'].apply(lambda x: [item['qty'] for item in json.loads(x)] if x is not None else [])

In [10]:
for c in df_product_list:
    # df_test[c] = 0
    df_full[c] = df_full['product'].apply(lambda x: 1 if c in x else 0)
    
    # print(f"c = {c}   {asd}")
    # if df_test["product_json"].str.contains(c):
    #     df_test[c] = 1
    # else:
    #     df_test[c] = 0

In [11]:
df_full.head()

,payment_date,ent,center,product_json,total_amount,discount,paid_amount,trans_origin_type,product,qty,...,XC4CCE,XC4CCQ,XC4CCW,XCECC4,XCECCJ,XCECCQ,XCJCC3,XCJCCZ,XCQCC4,XP9LI
0,2021-01-01 00:00:00.000,TCC43RQQZ3B,TKC4CJ,"[{""product"":""1ECZ"",""qty"":1},{""product"":""8CEC4Q...",1627000.0,0.0,0,online,"[1ECZ, 8CEC4Q]","[1, 1]",...,0,0,0,0,0,0,0,0,0,0
1,2021-01-01 00:00:00.000,TCC4Q4JZ4Q0,TKCEWQ,"[{""product"":""4E4W"",""qty"":2},{""product"":""DC4CC4...",594000.0,0.0,0,online,"[4E4W, DC4CC4]","[2, 1]",...,0,0,0,0,0,0,0,0,0,0
2,2021-01-01 00:00:00.000,TCC4RJ11QE7,TKC14E,"[{""product"":""IC4CC4"",""qty"":1}]",468000.0,0.0,0,online,[IC4CC4],[1],...,0,0,0,0,0,0,0,0,0,0
3,2021-01-01 00:00:00.000,TCC434QRCJY,TKC14R,"[{""product"":""5C4CC4"",""qty"":1}]",341000.0,0.0,0,online,[5C4CC4],[1],...,0,0,0,0,0,0,0,0,0,0
4,2021-01-01 00:00:00.000,TCC43RRCE10,TKC11Z,"[{""product"":""5C4C4Q"",""qty"":1}]",189000.0,0.0,0,online,[5C4C4Q],[1],...,0,0,0,0,0,0,0,0,0,0


Check missing values in total_amount, discount, paid_amount

In [13]:
print('total_amount', df_full.total_amount.isna().sum())
print('discount', df_full.discount.isna().sum())
print('paid_amount', df_full.paid_amount.isna().sum())

total_amount 0
discount 140
paid_amount 0


In [14]:
# fillna with 0
df_full['total_amount'] = df_full['total_amount'].fillna(0)
df_full['discount'] = df_full['discount'].fillna(0)
df_full['paid_amount'] = df_full['paid_amount'].fillna(0)

In [15]:
# Recalculate paid_amount = total_amount - discount
df_full['paid_amount'] = df_full['total_amount'] - df_full['discount']
df_full.head()

,payment_date,ent,center,product_json,total_amount,discount,paid_amount,trans_origin_type,product,qty,...,XC4CCE,XC4CCQ,XC4CCW,XCECC4,XCECCJ,XCECCQ,XCJCC3,XCJCCZ,XCQCC4,XP9LI
0,2021-01-01 00:00:00.000,TCC43RQQZ3B,TKC4CJ,"[{""product"":""1ECZ"",""qty"":1},{""product"":""8CEC4Q...",1627000.0,0.0,1627000.0,online,"[1ECZ, 8CEC4Q]","[1, 1]",...,0,0,0,0,0,0,0,0,0,0
1,2021-01-01 00:00:00.000,TCC4Q4JZ4Q0,TKCEWQ,"[{""product"":""4E4W"",""qty"":2},{""product"":""DC4CC4...",594000.0,0.0,594000.0,online,"[4E4W, DC4CC4]","[2, 1]",...,0,0,0,0,0,0,0,0,0,0
2,2021-01-01 00:00:00.000,TCC4RJ11QE7,TKC14E,"[{""product"":""IC4CC4"",""qty"":1}]",468000.0,0.0,468000.0,online,[IC4CC4],[1],...,0,0,0,0,0,0,0,0,0,0
3,2021-01-01 00:00:00.000,TCC434QRCJY,TKC14R,"[{""product"":""5C4CC4"",""qty"":1}]",341000.0,0.0,341000.0,online,[5C4CC4],[1],...,0,0,0,0,0,0,0,0,0,0
4,2021-01-01 00:00:00.000,TCC43RRCE10,TKC11Z,"[{""product"":""5C4C4Q"",""qty"":1}]",189000.0,0.0,189000.0,online,[5C4C4Q],[1],...,0,0,0,0,0,0,0,0,0,0


In [18]:
# save to parquet
df_full.to_parquet('data/df_full.parquet')